In [1]:
import os

In [2]:
%pwd #projec tworking directory

'/home/suganya/Documents/Text-Summarization-Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/suganya/Documents/Text-Summarization-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass (frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path


In [6]:
from textSummarizer.contants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # This CALLS the create_directories function (defined elsewhere)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [8]:
import os 
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size
from pathlib import Path
#from textSummarizer.entity import (DataIngestionConfig)

In [11]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename =self.config.local_data_file
            )
            logger.info(f"(filename) download with following info: \n{headers}")
        else:
            logger.info(f"File already exist of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
            zip_file_path:str
            Extract zip file into data directory
            Funtions Returns none
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file , 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-10-03 16:44:57,966: INFO: common: yaml file: /home/suganya/Documents/Text-Summarization-Project/config/config.yaml loaded successfully]
[2025-10-03 16:44:57,970: INFO: common: yaml file: /home/suganya/Documents/Text-Summarization-Project/params.yaml loaded successfully]
[2025-10-03 16:44:57,973: INFO: common: created directory: artifacts]
[2025-10-03 16:44:57,974: INFO: common: created directory: artifacts/data_ingestion]
[2025-10-03 16:45:03,085: INFO: 3409366772: (filename) download with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 23F0:33560C:14553B:2FF712:68DFB033
Accept-Ranges: bytes
Date: Fri, 03 Oct 2025 